<a href="https://colab.research.google.com/github/irshandyaditya/machine_learning/blob/main/P10/Tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import time

In [2]:
# 1. Mengunduh dan mempersiapkan dataset tanpa as_supervised=True
dataset, info = tfds.load('tiny_shakespeare', with_info=True)
train_data = dataset['train']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.KOS6KN_1.0.0/tiny_shakespeare-train.tfrecord*.…

Generating validation examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.KOS6KN_1.0.0/tiny_shakespeare-validation.tfrec…

Generating test examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.KOS6KN_1.0.0/tiny_shakespeare-test.tfrecord*..…

Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [3]:
# Menggabungkan semua contoh teks menjadi satu teks panjang
text = ""
for example in train_data:
    text += example['text'].numpy().decode('utf-8')

In [4]:
# 2. Memetakan karakter ke indeks dan sebaliknya
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = {i: u for i, u in enumerate(vocab)}

# Mengonversi teks menjadi ID integer berdasarkan karakter
text_as_int = [char2idx[c] for c in text]

In [5]:
# 3. Membuat dataset dengan sequence karakter
seq_length = 100  # panjang sequence
examples_per_epoch = len(text) // seq_length

In [6]:
# Membuat sequence karakter menjadi batch
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [7]:
# Membagi sequence menjadi input dan target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [8]:
dataset = sequences.map(split_input_target)

In [9]:
# Mengacak dan membuat batch untuk pelatihan
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
# 4. Definisikan model berbasis RNN (GRU)
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(MyModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)

        if states is None:
            gru_output = self.gru(x, training=training)
        else:
            gru_output = self.gru(x, initial_state=states, training=training)

        x = gru_output[0]
        states = gru_output[1] if return_state else None
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [15]:
# Custom training class dengan train_step
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return {'loss': loss}

In [16]:
# Inisialisasi model dan kompilasi
embedding_dim = 256
rnn_units = 1024
vocab_size = len(vocab)

In [17]:
model = CustomTraining(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [19]:
# 5. Custom training loop
EPOCHS = 10
checkpoint_prefix = "./checkpoints/ckpt"
mean_loss = tf.keras.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()
    mean_loss.reset_state()

    for batch_n, (inp, target) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean_loss.update_state(logs['loss'])

        # Cetak loss setiap 50 batch
        if batch_n % 50 == 0:
            print(f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}")

    # Simpan model (checkpoint) setiap 5 epoch
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    # Tampilkan loss setelah setiap epoch
    print(f'\nEpoch {epoch+1} Loss: {mean_loss.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_" * 80)

# Simpan model setelah selesai pelatihan
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.1743
Epoch 1 Batch 50 Loss 2.5150
Epoch 1 Batch 100 Loss 2.2408
Epoch 1 Batch 150 Loss 2.0834

Epoch 1 Loss: 2.5710
Time taken for 1 epoch 921.93 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.0527
Epoch 2 Batch 50 Loss 1.9527
Epoch 2 Batch 100 Loss 1.8477
Epoch 2 Batch 150 Loss 1.7411

Epoch 2 Loss: 1.8898
Time taken for 1 epoch 921.92 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6935
Epoch 3 Batch 50 Loss 1.6593
Epoch 3 Batch 100 Loss 1.5948
Epoch 3 Batch 150 Loss 1.5324

Epoch 3 Loss: 1.6219
Time taken for 1 epoch 921.92 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.5373
Epoch 4 Batch 50 Loss 1.4763
Epoch 4 Batch 100 Loss 1.4653
Epoch 4 Batch 150 Loss 1.4882

Epoch 4 Loss: 1.4779
Time taken for 1 epoch 888.98 sec
_________________________________________________________________

ValueError: The filename must end in `.weights.h5`. Received: filepath=./checkpoints/ckpt